In [20]:
import tensorflow as tf
import tensorlayer as tl
import numpy as np
tl.logging.set_verbosity(tl.logging.DEBUG)

In [2]:
X_train,y_train,X_val,y_val,X_test,y_test=tl.files.load_mnist_dataset(shape=(-1,784))

[TL] Load or Download MNIST > data/mnist
[TL] data/mnist/train-images-idx3-ubyte.gz
[TL] data/mnist/t10k-images-idx3-ubyte.gz


In [4]:
##讲数据分为训练集合（training set），验证集(validcation set)和测试集(testing set)把每一个手写的数字用一行向量784个进行表示

In [3]:
print(X_train.shape,y_train.shape)##代表有50000个图片作为训练集合

(50000, 784) (50000,)


In [4]:
print(X_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
print(X_val.shape,y_val.shape)##1000作为验证集合

(10000, 784) (10000,)


In [6]:
print(X_test.shape,y_test.shape)##1000作为测试集合

(10000, 784) (10000,)


In [7]:
print(y_train)

[5 0 4 ... 8 4 8]


In [8]:
##定义一个网络

In [9]:
ni=tl.layers.Input([None,784])##其实代表了batch 一批几个这个none 784代表一个数字 将一个数字拉成1行784列
nn=tl.layers.Dropout(keep=0.8)(ni)
nn=tl.layers.Dense(n_units=800,act=tf.nn.relu)(nn)
nn=tl.layers.Dropout(keep=0.5)(nn)
nn=tl.layers.Dense(n_units=800,act=tf.nn.relu)(nn)
nn=tl.layers.Dropout(keep=0.5)(nn)
nn=tl.layers.Dense(n_units=10,act=None)(nn)
network=tl.models.Model(inputs=ni,outputs=nn,name="mlp_simp")

[TL] Input  _inputlayer_1: [None, 784]
[TL] Dropout dropout_1: keep: 0.800000 
[TL] Dense  dense_1: 800 relu
[TL] Dropout dropout_2: keep: 0.500000 
[TL] Dense  dense_2: 800 relu
[TL] Dropout dropout_3: keep: 0.500000 
[TL] Dense  dense_3: 10 No Activation


In [10]:
##定义准确率函数

In [11]:
def acc(_logits,y_batch):
    return tf.reduce_mean(
    tf.cast(tf.equal(tf.argmax(_logits,1),tf.convert_to_tensor(y_batch,tf.int64)),tf.float32,name='accuracy')
    )

In [12]:
##tf.euqal 
##x = tf.constant([2, 4])
##y = tf.constant(2)
##tf.math.equal(x, y) ==> array([True, False])
##tf.equal(tf.argmax(_logits,1),tf.convert_to_tensor(y_batch,tf.int64)) 是预估值和标签值相等判断后输出的列表
##tf.reduce_mean求均值

In [13]:
print(network)

mlp_simp(
  (_inputlayer_1): Input(shape=[None, 784], name='_inputlayer_1')
  (dropout_1): Dropout(keep=0.8, name='dropout_1')
  (dense_1): Dense(n_units=800, relu, in_channels='784', name='dense_1')
  (dropout_2): Dropout(keep=0.5, name='dropout_2')
  (dense_2): Dense(n_units=800, relu, in_channels='800', name='dense_2')
  (dropout_3): Dropout(keep=0.5, name='dropout_3')
  (dense_3): Dense(n_units=10, No Activation, in_channels='800', name='dense_3')
)


In [14]:
##训练模型


In [15]:
tl.utils.fit(
    network, train_op=tf.optimizers.Adam(learning_rate=0.0001), cost=tl.cost.cross_entropy, X_train=X_train,
    y_train=y_train, acc=acc, batch_size=256, n_epoch=20, X_val=X_val, y_val=y_val, eval_train=True,
    tensorboard_dir='./tb_loog'
)

[TL] Setting up tensorboard ...
[TL] [!] ./tb_loog exists ...
[TL] Start training the network ...
[TL] Epoch 1 of 20 took 10.085436s
[TL]    train loss: 0.410896
[TL]    train acc: 0.883033
[TL] Epoch 5 of 20 took 9.307992s
[TL]    train loss: 0.191777
[TL]    train acc: 0.942488
[TL] Epoch 10 of 20 took 7.562018s
[TL]    train loss: 0.123512
[TL]    train acc: 0.962400
[TL] Epoch 15 of 20 took 7.625322s
[TL]    train loss: 0.088859
[TL]    train acc: 0.972857
[TL] Epoch 20 of 20 took 7.713785s
[TL]    train loss: 0.068076
[TL]    train acc: 0.979607
[TL] Total training time: 126.768179s


In [23]:
tl.utils.test(network, acc, X_test, y_test, batch_size=None, cost=tl.cost.cross_entropy)

[TL] Start testing the network ...


<tf.Tensor: id=51735, shape=(), dtype=float32, numpy=0.9743>

In [24]:
tl.utils.test?


In [27]:
_logits = tl.utils.predict(network, X_test)
print(_logits)
y_pred = np.argmax(_logits, 1)
print(y_pred)
tl.utils.evaluation(y_test, y_pred, n_classes=10)

tf.Tensor(
[[-2.271208   -2.6272442   2.2785277  ... 11.591954   -3.7504613
   0.34867537]
 [-1.614523    2.086506   10.253306   ... -5.161183    0.20881243
  -8.756842  ]
 [-4.453318    8.496254   -0.20584133 ...  0.87646633 -0.85555
  -3.5992785 ]
 ...
 [-7.637353   -3.6960683  -4.9947667  ... -0.18907967  0.19720453
   3.927492  ]
 [-0.7638174  -2.0758698  -6.183501   ... -4.523805    3.2531538
  -3.380673  ]
 [ 0.2428626  -3.8809628   0.7159096  ... -7.7537045  -3.403752
  -5.6272707 ]], shape=(10000, 10), dtype=float32)
[7 2 1 ... 4 5 6]
[TL] confusion matrix: 
[[ 971    0    0    0    0    1    4    1    2    1]
 [   0 1126    2    1    0    1    3    0    2    0]
 [   7    0 1002    3    1    0    3    7    9    0]
 [   1    0    4  984    0    8    0    5    7    1]
 [   2    0    2    0  951    0    6    3    2   16]
 [   4    0    0    7    1  870    6    0    3    1]
 [   5    3    0    1    5    3  939    0    2    0]
 [   2    8   11    2    1    1    0  993    0   10]
 [ 

(array([[ 971,    0,    0,    0,    0,    1,    4,    1,    2,    1],
        [   0, 1126,    2,    1,    0,    1,    3,    0,    2,    0],
        [   7,    0, 1002,    3,    1,    0,    3,    7,    9,    0],
        [   1,    0,    4,  984,    0,    8,    0,    5,    7,    1],
        [   2,    0,    2,    0,  951,    0,    6,    3,    2,   16],
        [   4,    0,    0,    7,    1,  870,    6,    0,    3,    1],
        [   5,    3,    0,    1,    5,    3,  939,    0,    2,    0],
        [   2,    8,   11,    2,    1,    1,    0,  993,    0,   10],
        [   4,    0,    2,    7,    3,    4,    5,    4,  941,    4],
        [   4,    6,    2,   10,    8,    2,    1,    4,    6,  966]]),
 array([0.98080808, 0.98858648, 0.97423432, 0.97185185, 0.97438525,
        0.97643098, 0.97558442, 0.97114914, 0.9661191 , 0.96215139]),
 0.9743,
 0.9741301006958564)

In [31]:

X_test.shape

(10000, 784)

In [32]:
_logits.shape

TensorShape([10000, 10])

In [30]:
_logits[1]

<tf.Tensor: id=51769, shape=(10,), dtype=float32, numpy=
array([-1.614523  ,  2.086506  , 10.253306  ,  2.7536051 , -7.9567223 ,
       -1.6947469 , -2.5419915 , -5.161183  ,  0.20881243, -8.756842  ],
      dtype=float32)>

In [33]:
y_pred = np.argmax(_logits, 1)

In [34]:
y_pred.shape

(10000,)

In [38]:
y_pred[8]

6

In [39]:
tl.utils.evaluation(y_test, y_pred, n_classes=10)

[TL] confusion matrix: 
[[ 971    0    0    0    0    1    4    1    2    1]
 [   0 1126    2    1    0    1    3    0    2    0]
 [   7    0 1002    3    1    0    3    7    9    0]
 [   1    0    4  984    0    8    0    5    7    1]
 [   2    0    2    0  951    0    6    3    2   16]
 [   4    0    0    7    1  870    6    0    3    1]
 [   5    3    0    1    5    3  939    0    2    0]
 [   2    8   11    2    1    1    0  993    0   10]
 [   4    0    2    7    3    4    5    4  941    4]
 [   4    6    2   10    8    2    1    4    6  966]]
[TL] f1-score        : [0.98080808 0.98858648 0.97423432 0.97185185 0.97438525 0.97643098
 0.97558442 0.97114914 0.9661191  0.96215139]
[TL] f1-score(macro) : 0.974130
[TL] accuracy-score  : 0.974300


(array([[ 971,    0,    0,    0,    0,    1,    4,    1,    2,    1],
        [   0, 1126,    2,    1,    0,    1,    3,    0,    2,    0],
        [   7,    0, 1002,    3,    1,    0,    3,    7,    9,    0],
        [   1,    0,    4,  984,    0,    8,    0,    5,    7,    1],
        [   2,    0,    2,    0,  951,    0,    6,    3,    2,   16],
        [   4,    0,    0,    7,    1,  870,    6,    0,    3,    1],
        [   5,    3,    0,    1,    5,    3,  939,    0,    2,    0],
        [   2,    8,   11,    2,    1,    1,    0,  993,    0,   10],
        [   4,    0,    2,    7,    3,    4,    5,    4,  941,    4],
        [   4,    6,    2,   10,    8,    2,    1,    4,    6,  966]]),
 array([0.98080808, 0.98858648, 0.97423432, 0.97185185, 0.97438525,
        0.97643098, 0.97558442, 0.97114914, 0.9661191 , 0.96215139]),
 0.9743,
 0.9741301006958564)

In [40]:
network.save_weights('helloword.h5')

[TL] [*] Saving TL weights into helloword.h5
[TL] [*] Saved
